---

<div align="center">

# MHCGlobe & MHCPerf
    
[![DOI:00.0000/2022.01.20.000000](http://img.shields.io/badge/DOI-00.0000/0000.00.00.000000-B31B1B.svg)](https://mhcglobe)

</div>

-----------

**Motivation:** Computational tools that predict peptide binding by major histocompatibility complex (MHC) proteins play an essential role in current approaches to harness adaptive immunity to fight viral pathogens and cancers. However, there are >22,000 known class-I MHC allelic variants, and it is unknown how well binding preferences are predicted for most alleles. We introduce a machine learning framework that enables state-of-the-art MHC binding prediction along with per-allele estimates of predictive performance. 

**Results:** We demonstrate stark disparities in how much binding data are associated with HLA alleles of individuals across racial and ethnic groups. Pan-MHC modeling mitigates some of these disparities when predicting MHC-peptide binding, and we devise a strategy to begin to address remaining inequities by leveraging our per-allele predictions of performance. The approaches introduced here further the development of equitable MHC binding models, which are necessary to understand adaptive immune response and to design effective personalized immunotherapies in genetically diverse individuals.

# MHCGlobe use cases
-----------

### 1. Predict peptide-MHC binding affinity

MHCGlobe predicts a binding score, and both the binding score and binding affinity (nM), `mhcglobe_score` and `mhcglobe_affinity`, respectively.

`MHCGlobe(train_type='full')` returns the MHCGlobe model trained on the full database of MHC-peptide binding instances in the present study, including both human and non-human pMHC data. 

In [1]:
import os
import pandas as pd
import sys
sys.path.append('./mhcglobe/')
sys.path.append('./mhcglobe/mhcperf')
from mhcglobe import MHCGlobe

# Load MHCGlobe class object containing the fully trained model.
mhcglobe = MHCGlobe(train_type='full')

# Load binding data as CSV. Required columns are `allele` and `peptide`.
prediction_cols = ['allele', 'peptide']
pMHC_data = pd.read_csv('example_binding_data.csv', usecols=prediction_cols)

# Predict peptide-MHC binding with MHCGlobe
mhcglobe.predict_on_dataframe(pMHC_data)

,allele,peptide,mhcglobe_affinity,mhcglobe_score
0,HLA-B*15:01,AQMWSLMYF,64.439289,0.614990
1,HLA-A*33:01,DIDILQTNSR,396.716782,0.447011
2,HLA-B*38:01,QPKKAAAAL,20009.974714,0.084641
3,HLA-A*31:01,FLALGFFLR,117.473925,0.559490
4,HLA-B*07:02,RPQKRPSCI,104.042288,0.570712
...,...,...,...,...
95,HLA-A*30:01,KAFNHASVK,118.069387,0.559023
96,HLA-A*02:01,WMMAMKYPI,61.744092,0.618939
97,HLA-A*02:03,FLMGFNRDV,30.050617,0.685494
98,HLA-A*02:01,LLSTTEWQV,91.183992,0.582905


### 2. Re-train MHCGlobe

MHCGlobe contains an ensemble of deep neural network models, which can easily trained on user defined peptide-MHC binding data. We recomend training MHCGlobe using the initialized weights and biases that were used in the paper for comparable results. The recomended initalized MHCGlobe model can be accessed using `MHCGlobe(train_type='init')`.


In [ ]:
# Load example training data.
training_cols = ['allele', 'peptide', 'measurement_value', 'measurement_inequality']
pMHC_data = pd.read_csv('example_binding_data.csv', usecols=training_cols)

# Re-train
new_mhcglobe_path = '/tf/local/mhcglobe_models/mhcglobe_example'
new_mhcglobe = MHCGlobe('init').train_ensemble(
        df_train          = pMHC_data,
        new_mhcglobe_path = new_mhcglobe_path,
        verbose           = 0)

In [ ]:
# Load user trained MHCGlobe
new_mhcglobe = MHCGlobe(train_type='example', new_mhcglobe_path=new_mhcglobe_path)

# Predict peptide-MHC binding with user trained MHCGlobe.
new_mhcglobe.predict_on_dataframe(pMHC_data.loc[:, prediction_cols])

# MHCPerf use cases
__________

The fully trained MHCPerf will be used to predict allele-level performance of a given MHCGlobe model based on its training data featurized.

### 1. Estimate allele-level PPV for query MHC alleles given a pMHC binding dataset.

MHCPerf features will be created for all query alleles based on this data count dictionary, relating the binding dataset to each of the query alleles.

In [2]:
#sys.path.append('./mhcperf_BEST/') # make sure mhcglobe doesn't get trained with mhcperf scripts.

import mhcperf

query_alleles = ['HLA-B*15:13','HLA-A*30:03','HLA-C*14:02']
mhcglobe_training_data = '/tf/fairmhc/data_git_ignore/data_preprocessed_22Oct2020.csv'

# Load MHCPerf model
mhcperf_model = mhcperf.model()

# MHCPerf inputs
perf_features = mhcperf.featurize_from_binding(mhcglobe_training_data, query_alleles)

# Predict PPV for each MHC allele in `query_alleles`
query_allele_ppv_est = mhcperf_model.predict_ppv(perf_features)

# Show allele-level PPV estimates by MHCPerf
pd.DataFrame(zip(query_alleles, query_allele_ppv_est), columns=['allele', 'PPV_est'])

,allele,PPV_est
0,HLA-B*15:13,0.761318
1,HLA-A*30:03,0.563628
2,HLA-C*14:02,0.629275


### 2. MHCPerf Hyperparameter Tuning and Training

MHCPerf is a single neural network which can be retained on new observations of MHCGlobe PPV performance in response to changes to the MHCGlobe training set. MHCPerf re-training repeats hyperparameter optimization using the grid search algorithm. This procedure can be run in the notebook (as shown below) or run in parallel in the backgroun using the following script call. 

    $ python3 /tf/mhcglobe/mhcperf/mhcperf.py {df_train_path} {new_mhcperf_path}

In [3]:
# Training Data for MHCPerf
# Feature column meansings are shown in Supplementary Figure 1.

df_train_path  = '/tf/fairmhc/mhcglobe/data/mhcperf_data/mhcperf_data.csv'
df_train = pd.read_csv(df_train_path)
df_train.head()

,allele,fold,trial,left_out_alleles,model_id,PPV,data_aa_pos_1,data_aa_pos_2,data_aa_pos_3,data_aa_pos_4,...,N10_data,dist_bin_0.0,dist_bin_0.1,dist_bin_0.2,dist_bin_0.3,dist_bin_0.4,dist_bin_0.5,dist_bin_0.6,dist_bin_0.7,data_size
0,HLA-B*08:01,1,7,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_37,0.231819,1010678,17258,140937,271779,...,21.0,0.0,0.0,87521.0,100971.0,387642.0,407716.0,26326.0,502.0,1010678
1,HLA-B*08:01,1,9,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_39,0.240697,1006549,17258,136912,267754,...,13.0,0.0,0.0,83392.0,100971.0,387642.0,407716.0,26326.0,502.0,1006549
2,HLA-B*08:01,1,8,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_38,0.251665,1006653,17258,136912,267754,...,1247.0,0.0,0.0,83496.0,100971.0,387642.0,407716.0,26326.0,502.0,1006653
3,HLA-B*08:01,1,4,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_34,0.252187,1013154,17258,143370,274212,...,33460.0,0.0,987.0,89010.0,100971.0,387642.0,407716.0,26326.0,502.0,1013154
4,HLA-B*08:01,1,6,"HLA-B*08:01,HLA-B*08:02,HLA-B*42:02,HLA-B*42:0...",LO_model_36,0.257018,1012167,17258,142426,273268,...,7.0,0.0,0.0,89010.0,100971.0,387642.0,407716.0,26326.0,502.0,1012167


In [4]:
# Re-train MHCPerf, includes hyperparameter tuning (48 combinations).

new_mhcperf_path = '/tf/local/mhcperf_models/mhcperf_example'

if not os.path.exists(new_mhcperf_path):
    # Train new MHCPerf
    new_mhcperf  = mhcperf.train(
        df_train_path  = df_train_path,
        model_savepath = new_mhcperf_path,
        verbose        = 0)
else:
    # Load New MHCPerf
    new_mhcperf = mhcperf.model(new_mhcperf_path, df_train_path)

### 3. Estimate allele-level PPV with new binding data.


In [14]:
import mhc_data
import featurize_mhcperf

def update_data_dict(allele_gets_data, data_to_add):
    ba_data_obj = mhc_data.pMHC_Data(only_EL=False, drop_duplicate_records=False)
    binding_data = ba_data_obj.positives
    data_dict = ba_data_obj.mk_data_dict(binding_data)
    
    data_dict[allele_gets_data] += data_to_add
    
    data_alleles, data_alleles_50 = ba_data_obj.get_data_alleles(data_dict)
    return data_dict, data_alleles

allele_gets_data = 'HLA-B*15:13'
data_to_add = 2000

data_dict_update, data_alleles_update = update_data_dict(allele_gets_data, data_to_add)

# update_features() updates an existing mhcperf featurized 
# df when adding more data for a single allele, without
# Recomputing the entire feature df for each query allele.

df_all_featurized_path = '/tf/mhcglobe/mhcglobe_ppv/mhcglobe/src/df_all_AD.csv.gz'
df_all = pd.read_csv(df_all_featurized_path)

classical_hlas = list(df_all.loc[[a[:5] in ['HLA-A', 'HLA-B', 'HLA-C'] for a in df_all.allele],:].allele.unique())

df_all_features = featurize_mhcperf.update_features(
            df_all,
            allele_gets_data,
            data_to_add,
            data_alleles_update,
            classical_hlas,
            data_dict_update
)

mhcperf_model = mhcperf.model()
ppv_estimates = mhcperf_model.predict_ppv(df_all_features)

pd.DataFrame(zip(df_all.allele, ppv_estimates), columns=['allele', 'PPV_est'])

12
2012
